In [32]:
# Config Section: inital values, constants, libraries etc

import math
from datetime import date, datetime
from dateutil import parser

import plotly
plotly.tools.set_credentials_file(username='pmoolrajani', api_key='lqy9gutcnu')
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go

# setting constants
__EVENT_TYPES = {
    'incidents':{'risk': 0.5, 'sla':3},
    'patching':{'risk': 0.3, 'sla': 7},
    'misc':{'risk': 0.2, 'sla': 5}
}
__SCORE_MAX = 1000
__SCORE_MIN = 0
__NO_EVENTS_FRACTION = 0.90
__TIME_INTERVAL = 7
__NUM_DAYS = 30


# setting initial values
score_avg_company = 620
score_emp = score_avg_company
risk = 0.5

In [33]:
# Logic Section - normal user   

# setting initial values
score_avg_company = 620
score_emp = score_avg_company


# creating dictionary of events from the input csv file
dict_events = {}
file_events = open('data/scoring-replay-attack.csv').readlines()


dict_events = {}
for line in file_events:
    
    event_details = line.split(',')
    if event_details[0] != 'event_date':
        event_date = parser.parse(event_details[0].strip())
        event_type = event_details[1].strip()
        num_events_recv = event_details[2].strip()
        num_events_resp = event_details[3].strip()
        num_events_sla_breach = event_details[4].strip()

        dict_events[event_date] = {}
        dict_events[event_date]['event_type'] = event_type
        dict_events[event_date]['num_events_recv'] = int(num_events_recv)
        dict_events[event_date]['num_events_resp'] = int(num_events_resp)
        dict_events[event_date]['num_events_sla_breach'] = int(num_events_sla_breach)
        

dict_scores = {}
for i in range(1, __NUM_DAYS, __TIME_INTERVAL):
    
    # calculating confidence intervals for employee score
    cpos = math.sqrt(__SCORE_MAX - score_emp)
    cneg = math.sqrt(score_emp)

    # calculating upper and lower limit of security score of an employee for __TIME_INTERVAL
    score_cpos = score_emp + cpos
    score_cneg = score_emp - cneg
    
    print ('\n\n')
    print ('TIME_INTERVAL: '+str(datetime(2016, 8, i, 0, 0)))
    print ('SCORE: '+str(score_emp))
    print ('C+ve: '+str(score_cpos)+'  |  '+'C-ve: '+str(score_cneg))
    print ('')
    no_events_counter = 0
    for j in range(i, i + __TIME_INTERVAL):
        if j > 31:
            break
            
        date_counter = datetime(2016, 8, j, 0, 0)
        
        if dict_events[date_counter]['num_events_recv'] == 0:
            
            print ('#days_left: '+str(__TIME_INTERVAL - (j-i)))
            print ('#numerator: '+str(score_cpos - score_emp))
            
            score_emp += (score_cpos - score_emp)*__NO_EVENTS_FRACTION/(__TIME_INTERVAL - (j-i))
        
        print ('\t'+str(date_counter))
        print ('\t\t'+'score_before: '+str(score_emp))
        print ('\t\t'+'events: '+'resp - '+str(dict_events[date_counter]['num_events_resp'])\
               +', recv - '+str(dict_events[date_counter]['num_events_recv'])\
               +', sla_breach - '+str(dict_events[date_counter]['num_events_sla_breach'])) 
        
        for k in range(0, dict_events[date_counter]['num_events_resp']):
            score_emp = score_emp + (score_cpos - score_emp)*(risk)
        for k in range(0, dict_events[date_counter]['num_events_sla_breach']):
            score_emp = score_emp - (score_emp - score_cneg)*(risk)
        
        print ('\t\t'+'score_after: '+str(score_emp))
        
        dict_scores[date_counter] = score_emp
        
# plotting score on graph
x_axis = []
y_axis = []
for day in dict_scores:
    x_axis.append(str(day))
    y_axis.append(dict_scores[day])

trace0 = go.Bar(x=x_axis, y=y_axis)





TIME_INTERVAL: 2016-08-01 00:00:00
SCORE: 620
C+ve: 639.4935886896179  |  C-ve: 595.1002008040225

	2016-08-01 00:00:00
		score_before: 620
		events: resp - 50, recv - 50, sla_breach - 0
		score_after: 639.4935886896178
	2016-08-02 00:00:00
		score_before: 639.4935886896178
		events: resp - 50, recv - 50, sla_breach - 0
		score_after: 639.4935886896178
	2016-08-03 00:00:00
		score_before: 639.4935886896178
		events: resp - 50, recv - 50, sla_breach - 0
		score_after: 639.4935886896178
	2016-08-04 00:00:00
		score_before: 639.4935886896178
		events: resp - 50, recv - 50, sla_breach - 0
		score_after: 639.4935886896178
	2016-08-05 00:00:00
		score_before: 639.4935886896178
		events: resp - 50, recv - 50, sla_breach - 0
		score_after: 639.4935886896178
	2016-08-06 00:00:00
		score_before: 639.4935886896178
		events: resp - 50, recv - 50, sla_breach - 0
		score_after: 639.4935886896178
	2016-08-07 00:00:00
		score_before: 639.4935886896178
		events: resp - 50, recv - 50, sla_breach - 0


In [34]:
data = [trace0]
print ("Replay Attack - 1500+ events in 30 days")
py.iplot(data, filename='scoring-replay-attack')


Replay Attack - 1500+ events in 30 days
